In [14]:
import os
import json
import math

import sys
sys.path.append("../utils")
from minimal_image import get_compact

from skimage.io import MultiImage

import matplotlib.pyplot as plt

import numpy as np

import cv2 as cv

from tqdm import tqdm

In [2]:
with open("../input/compact_representation.json", "r") as file:
    compact_representation = json.load(file)

In [10]:
data_dir = "../input/prostate-cancer-grade-assessment/train_images/"
save_dir = "/datasets/panda/train_64_100"

tile_size = 64
num_tiles = 100

In [11]:
def tile(img, sz=128, N=16):
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    
    h, w = img.shape[:2]
    
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)

    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    
    img = img[idxs]
    s = int(np.sqrt(N))
    result = np.empty([sz * s, sz * s, 3], dtype=np.uint8)
    for i in range(len(img)):
        x = i % s
        y = i // s
        result[y * sz:(y + 1) * sz, x * sz:(x + 1) * sz] = img[i]
    
    return result

In [15]:
os.makedirs(save_dir, exist_ok=True)

for key, item in tqdm(compact_representation.items()):
    img = MultiImage(os.path.join(data_dir, key))[-1]
    img = tile(img, tile_size, num_tiles)
    
    save_path = os.path.join(save_dir, os.path.splitext(key)[0] + ".png")
    
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    cv.imwrite(save_path, img)

100%|██████████| 10616/10616 [08:13<00:00, 21.52it/s]
